In [ ]:
rm(list=ls())
libraries = c("dplyr","magrittr","tidyr","reshape2","ggplot2","ggpubr","extrafont","openxlsx","readxl","RColorBrewer","MetBrewer","zoo","RVAideMemoire"
              ,"writexl","gridExtra","MASS","readr","stats","boot","pracma","stringr","mixdist","corpcor","rstan","bayesplot","graphics")
for(x in libraries) { library(x,character.only=TRUE,warn.conflicts=FALSE,quietly=TRUE) }

theme_set(theme_bw())
setwd(".../MPX_depletion_susceptibles")

In [ ]:
df <- read.csv("data/timeseries-country-confirmed.csv") # retrieved as of 15 Sep 2022
#https://raw.githubusercontent.com/globaldothealth/monkeypox/main/timeseries-country-confirmed.csv

In [ ]:
df$Country %>% unique 
country <- df$Country %>% unique

In [ ]:
source("src/R/colour.R")

In [ ]:
source("src/Stan/growth_estim.stan")

In [ ]:
source("src/R/growth_estim_r.R") #excluding c_number[12,21,36,37,38,43];

In [ ]:
intro_date <- rep(0,length(range_summary1$Country_number))
numb_i <- range_summary1$Country_number %>% as.numeric()
for(i in 1:length(numb_i)){
    dff <- df %>% filter(Country==country[numb_i[i]])
  date_n<- as.Date(dff$Date) %>% as.numeric()
  dff %<>% cbind(date_n)
  dff <- dff %>% dplyr::select("Cases","Cumulative_cases","date_n")  
  row_sub = apply(dff, 1, function(row) all(row !=0 ))
  num <- grep("TRUE",row_sub)
  dff<- dff[row_sub,] 
  dff %<>% cbind(num)
  df_fin <- dff[!apply(as.data.frame(dff$Cumulative_cases), 1, function(row) all(row <=10 )),]
  intro_date[i] <- df_fin$date_n[1] 
}
intro_date[intro_date==0] <- NA
intro_date <- intro_date[!is.na(intro_date)]
intro_date <- intro_date-min(intro_date)
intro_df <- cbind(country[numb_i],intro_date)
colnames(intro_df) <- c("Country","intro_date")

In [ ]:
### dataframe for msm size
numb_i <- c(1:length(c_number))
numb_i <- numb_i[-c(12,14,16,22,24,25,29,30,32,44,45,50,51)]
range_summary <- matrix(rep(0,5*(length(c_number))), nrow = length(c_number), ncol = 5) %>% as.data.frame()
for(i in numb_i){
range_df <- read.csv(paste0("output/csv/country/",country[c_number[i]],"_growth.csv"))
n_of_dates <- nrow(range_df)
range_summary[i,] <- c(country[c_number[i]],c_number[i],range_df$lower[1], range_df$upper[n_of_dates],range_df$date[1])
    }
colnames(range_summary) <- c("Country","Country_number","lower","upper","peakdate")
latest_cases <- rep(0,length(c_number)) 
for(i in numb_i){
dfff <- df %>% filter(Country==country[c_number[i]]) %>% dplyr::select(Cumulative_cases) 
latest_n <- df %>% filter(Country==country[c_number[i]]) %>% dplyr::select(Cumulative_cases) %>% nrow()
latest_cases[i] <- dfff[latest_n,]
}
latest_cases %<>%  as.data.frame() 
range_summary %<>% cbind(latest_cases)
colnames(range_summary) <- c("Country","Country_number","lower","upper","peakdate","latest_cases")
range_summary[range_summary$latest_cases==0,] <-NA
#range_summary[range_summary$Country==0,] <-NA
range_summary %<>% drop_na()

pop_size <- c(26136558, 9115559, 11696761, 12011327, 215769215, 38445168, 52051411, 4050768, 10751718, 5846420, 5559000, 65580278, 84352236, 32452865, 10314829, 9607599, 5055121, 8655535, 60273038, 647588, 17215189, 5511411, 33961522, 37757348, 10196709, 2702749, 18961090, 2079544, 46793237, 10233038, 8789076, 68643511, 335184010)
msm_size <- c(NA,70985, 106336, NA, NA, 369500, NA, 29497, NA, 54723, 53118, 626948, 655740, NA, 102888, 53404, 46488, NA, 359315, 1749, 166872, 47483, NA, 134981, 107328, NA, 74916, 21591, 294028, 65632, 70229, 613658, 4503080)
a="Both limits";b="Only lower limit"
label <- c(b,a,a,a,a,a,b,b,b,a,b,a,a,b,a,b,b,a,a,a,a,a,a,b,a,b,b,a,a,a,a,a,a)
range_summary %<>% cbind(pop_size, msm_size, label)
upper1 <- ifelse(range_summary$label=="Only lower limit",0,range_summary$upper)
upper_forplot <- ifelse(range_summary$label=="Only lower limit",range_summary$upper,0)
perpop_lower <- as.numeric(range_summary$lower)/as.numeric(range_summary$pop_size)
perpop_upper <- as.numeric(range_summary$upper)/as.numeric(range_summary$pop_size)
range_summary %<>% cbind(upper1,upper_forplot,perpop_lower,perpop_upper) 


range_summary <- range_summary[!is.na(range_summary$msm_size),]
Country1 <- ifelse(range_summary$label=="Only lower limit",range_summary$Country,NA)
range_summary[range_summary==0] <-NA

permsm_lower <- as.numeric(range_summary$lower)/as.numeric(range_summary$msm_size)
permsm_upper <- as.numeric(range_summary$upper)/as.numeric(range_summary$msm_size)
range_summary %<>% cbind(permsm_lower,permsm_upper,Country1)
range_summary <- range_summary[order(permsm_lower,decreasing=TRUE),]
range_summary %<>% cbind(c("a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x"))
colnames(range_summary) <- c("Country","Country_number","lower","upper","peakdate","latest_cases","pop_size","msm_size","label","upper1","upper_forplot","perpop_lower","perpop_upper","permsm_lower","permsm_upper","Country1","Country2")
Country3 <- ifelse(range_summary$label=="Only lower limit",range_summary$Country2,NA)
latest_cases1 <-  ifelse(range_summary$label=="Only lower limit",range_summary$latest_cases,NA)
range_summary %<>% cbind(Country3,latest_cases1)

intro_date <- rep(0,length(range_summary$Country_number))
intro_date <- c()#rep(0,length(range_summary$Country_number))
numb_i <- range_summary$Country_number %>% as.numeric()

for(i in 1:length(numb_i)){
intro_df %>% as.data.frame() %>% filter(Country==country[numb_i[i]]) %>% dplyr::select(intro_date) %>% as.numeric() -> intro_date[i]
} 
intro_date[intro_date==0] <- NA
range_summary %<>% cbind(intro_date)
intro_date_loweronly <- ifelse(range_summary$label=="Only lower limit",range_summary$intro_date,NA)
msm_size1 <- ifelse(range_summary$label=="Only lower limit",range_summary$msm_size,NA)
mid_prop <- (range_summary$permsm_lower+range_summary$permsm_upper)/2 %>% as.numeric()
range_summary %<>% cbind(intro_date_loweronly,msm_size1,mid_prop)

In [ ]:
### calculate score

width_1 <- 1/100000
number_loweronly <- range_summary %>% filter(label=="Only lower limit") %>% nrow()
score <- rep(1,1/width_1)

for(i in 1:nrow(range_summary)){
width_score <- round(seq(range_summary$permsm_lower[i]/width_1,range_summary$permsm_upper[i]/width_1,1))
for(j in width_score) {score[j] = score[j] + 1}

width_score_1 <- round(seq(1+range_summary$permsm_upper[i]/width_1,1/width_1,1))
if(range_summary$label[i]=="Only lower limit"){
for(k in width_score_1) {score[k] = score[k] + 1}
} 
}
score <- score[!is.na(score)]

### specify the range of global estimates (threshold=12)

prop_hit <- rep(NA,1/width_1)
for(i in 1:(1/width_1 -1)){
if(score[i]>=12){
    prop_hit[i] = i*width_1
}
}

prop_hit <- prop_hit[!is.na(prop_hit)]
prop_length <- length(prop_hit)

### plot

options(repr.plot.width=12,repr.plot.height=8)
range_l <- prop_hit[1]
range_u <- prop_hit[prop_length]
df_score <- cbind(seq(1,0.03/width_1,1)*width_1,score[1:(0.03/width_1)]) %>% as.data.frame()
colnames(df_score) <- c("prop","score_value") 
df_score_plot <- df_score %>% ggplot() +
    annotate("rect", xmin = range_l*100, xmax = range_u*100, ymin = -Inf, ymax = Inf, fill = met.brewer("Hokusai2",n=2)[2], alpha = 0.15) +
    geom_line(aes(x=prop*100, y=score_value), colour=met.brewer("Troy",n=15)[6], size=1.2, alpha=2) + 
    geom_hline(yintercept=12, linetype="dashed", color = "#1380A1", size =0.7) + theme_bw(base_size = 18) + theme_bw(base_size = 18) + 
    scale_x_continuous(breaks=c(0, 0.5, 1, 1.5, 2, 2.5, 3),limits=c(0, 3)) +
    scale_color_manual(values=met.brewer("Cassatt2",8)[8]) + theme(axis.text.x = element_text(angle = 0, vjust = 0.5, hjust = 0.5),panel.grid.major=element_blank(), panel.grid.minor=element_blank()) +
    labs(x="Cumulative cases per MSM population at a peak of an epidemic [%]", y = "Number of countries")
df_score_plot
ggsave(df_score_plot,file = "output/fig/score_plot_country.png", width = 12, height = 8)

In [ ]:
options(repr.plot.width=18,repr.plot.height=8)
Country_label <- range_summary$Country
range_summary %>% ggplot() + 
　annotate("rect", ymin = range_l*100, ymax = range_u*100, xmin = -Inf, xmax = Inf, fill = met.brewer("Hokusai2",n=2)[2], alpha = 0.15) +
  geom_rect(aes(xmin= Country2, xmax=Country2, ymin=permsm_lower*100, ymax=permsm_upper*100, group=label, color=label, fill=label), size=3)  + geom_segment(aes(x=Country3,xend=Country3,y=permsm_upper*100,yend=2, color=label),size=1, linetype=2) +
  theme_bw(base_size = 20)+ theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1), legend.position="top", legend.justification='right', legend.title = element_blank(),panel.grid.major=element_blank(), panel.grid.minor=element_blank()) + scale_color_manual(values=c(met.brewer("Troy",n=15)[5],met.brewer("Troy",n=15)[10])) + 
  scale_fill_manual(values=c(met.brewer("Troy",n=15)[5],met.brewer("Troy",n=15)[10])) + scale_y_continuous(breaks=c(0,0.5, 1, 1.5, 2),limits=c(0,2)) +
  labs(x="", y = "Cumulative cases per MSM population\
at a peak of an epidemic [%]\
 ") + scale_x_discrete(labels = Country_label)
ggsave(file = "output/fig/incidenceprop_msm.jpeg", width = 18, height = 8)

In [ ]:
options(scipen=10000000)
options(repr.plot.width=10,repr.plot.height=8)
range_summary %>% ggplot() +
geom_point(aes(x=permsm_lower*100, y=as.numeric(msm_size)),size=4,colour=met.brewer("Austria",n=2)[1]) +
theme(legend.title = element_blank()) +
#scale_color_manual(values=c(met.brewer("Monet",n=9)[2],met.brewer("Monet",n=9)[8])) + scale_fill_manual(values=c(met.brewer("Monet",n=9)[2],met.brewer("Monet",n=9)[8])) +
#scale_x_continuous(breaks = seq(0, 5250, 500), limits=c(0, 5250)) + 
#scale_y_continuous(breaks = seq(0, 30000, 10000), limits=c(0, 30000)) + 
theme_bw(base_size = 22) + theme(legend.title = element_blank(),legend.text=element_text(size=20))  + theme(axis.text = element_text(size = 20)) +
labs(x="Cumulative cases\ 
per MSM population at a peak of an epidemic [%]", y = "MSM population size
 ")  

ggsave(file = "output/fig/correlation_msmsize.jpeg", width = 10, height = 8)

In [ ]:
spearman.ci(range_summary$permsm_lower, as.numeric(range_summary$msm_size), nrep = 1000, conf.level = 0.95)

In [ ]:
options(scipen=10000000)
options(repr.plot.width=10,repr.plot.height=8)
range_summary %>% ggplot() +
geom_point(aes(x=permsm_lower*100, y=intro_date),size=4,colour=met.brewer("Monet",n=9)[8]) +
theme(legend.title = element_blank()) +
#scale_color_manual(values=c(met.brewer("Monet",n=9)[2],met.brewer("Monet",n=9)[8])) + scale_fill_manual(values=c(met.brewer("Monet",n=9)[2],met.brewer("Monet",n=9)[8])) +
#scale_x_continuous(breaks = seq(0, 5250, 500), limits=c(0, 5250)) + 
#scale_x_continuous(breaks = seq(0, 30000, 10000), limits=c(0, 1000000)) + 
theme_bw(base_size = 22) + theme(legend.title = element_blank(),legend.text=element_text(size=20))  + theme(axis.text = element_text(size = 20)) +
labs(x="Cumulative cases\ 
per MSM population at a peak of an epidemic [%]", y = "Date of reporting the 10th case
 ")  

ggsave(file = "output/fig/correlation_date_point.jpeg", width = 10, height = 8)

In [ ]:
spearman.ci(range_summary$permsm_lower, range_summary$intro_date, nrep = 1000, conf.level = 0.95)

In [ ]:
options(repr.plot.width=15,repr.plot.height=8)
options(scipen=1000000)
total <- range_summary %>% ggplot(aes(y=as.numeric(pop_size))) + #geom_abline(slope=48000, intercept=0) +
geom_rect(aes(ymin= as.numeric(pop_size), ymax=as.numeric(pop_size), xmin=permsm_lower, xmax=permsm_upper, group=label, color=label, fill=label), size=1.2)  + 
geom_segment(aes(y=pop_size1,yend=pop_size1,x=permsm_upper,xend=0.0255, color=label),size=1, linetype=2) +
theme(legend.title = element_blank()) +
scale_color_manual(values=c(met.brewer("Monet",n=9)[2],met.brewer("Monet",n=9)[8])) + scale_fill_manual(values=c(met.brewer("Monet",n=9)[2],met.brewer("Monet",n=9)[8])) +
#scale_x_continuous(breaks = seq(0, 5250, 500), limits=c(0, 5250)) + scale_y_continuous(breaks = seq(0, max(intro_date), 5), limits=c(0, max(intro_date)+1)) + theme(legend.title = element_blank()) +
labs(x="Cumulative cases\ 
per total population (r~0)\
", y = "Relative date of introductions
 ") 

msm <- range_summary %>% ggplot(aes(y=as.numeric(msm_size))) +# geom_abline(slope=380, intercept=0) +
geom_rect(aes(ymin= as.numeric(msm_size), ymax=as.numeric(msm_size), xmin=permsm_lower, xmax=permsm_upper, group=label, color=label, fill=label), size=2)  + 
geom_segment(aes(y=msm_size1,yend=msm_size1,x=permsm_upper,xend=0.0255, color=label),size=1, linetype=2) +
theme(legend.title = element_blank()) +
scale_color_manual(values=c(met.brewer("Monet",n=9)[2],met.brewer("Monet",n=9)[8])) + scale_fill_manual(values=c(met.brewer("Monet",n=9)[2],met.brewer("Monet",n=9)[8])) +
#scale_x_continuous(breaks = seq(0, 5250, 500), limits=c(0, 5250)) + 
scale_y_continuous(breaks = seq(0, 400000, 50000), limits=c(0, 400000)) + theme(legend.title = element_blank()) +
labs(x="Cumulative cases\ 
per MSM population (r~0)\
", y = "Relative date of introductions
 ")  
ggarrange(total, msm, nrow=1,ncol=2,font.label=list(size=22),labels=c("A","B"),common.legend=TRUE)
#ggsave(file = "output/correlation.png", width = 15, height = 8)